## ChatBot
 - 트랜스포머를 활용한 챗봇
 - [(딥러닝을 이용한 자연어처리 입문)](https://wikidocs.net/89786)

### 1. 데이터 로드 및 탐색

In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import re

In [91]:
file_path = './data'
train_data = pd.read_csv(file_path + '/ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [92]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


In [93]:
# 결측치 확인
train_data.isna().sum()

Q        0
A        0
label    0
dtype: int64

In [94]:
# ? , . !과 같은 특수문자 전처리
# ex) 12시 땡! => 12시 땡 !
questions = []
for sentence in train_data['Q']:
    sentence = re.sub(r'([?.!,])', r' \1', sentence)    # \1, \2: 첫 번째, 두 번째 그룹
    sentence = sentence.strip()
    questions.append(sentence)
    
answers = []
for sentence in train_data['A']:
    sentence = re.sub(r'([?.!,])', r' \1', sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [95]:
print(questions[:7])
print(answers[:7])

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네', 'SD카드 망가졌어', 'SD카드 안돼']
['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .', '여행은 언제나 좋죠 .', '눈살이 찌푸려지죠 .', '다시 새로 사는 게 마음 편해요 .', '다시 새로 사는 게 마음 편해요 .']


### 2. 사전 생성
 - 질문과 답변 데이터로부터 사전 생성
 - 서브워드 토크나이저 중 SubwordTextEncoder 사용

In [96]:
# 토크나이저 정의 및 사전 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)    # 최대 사전 크기는 8192(=2**13)

In [97]:
# 사전 크기 확인
tokenizer.vocab_size

8176

In [98]:
# 만들어진 서브워드 확인
tokenizer.subwords[10:20]

['이_', '을_', '잘_', '도_', ' . ', '고_', '요', '것_', '많이_', '안_']

In [99]:
# SOS, EOS 토큰 만들어주기
# SOS, EOS 토큰에 정수 할당
START_TKN, END_TKN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# SOS, EOS 토큰만큼 사전 크기 +2로 늘려주기
VOCAB_SIZE = tokenizer.vocab_size + 2

In [100]:
print('시작 토큰 번호 :',START_TKN)
print('종료 토큰 번호 :',END_TKN)
print('단어 집합의 크기 :',VOCAB_SIZE)
# 추후 패딩토큰 추가
# 만든 사전은 save_to_file로 저장

시작 토큰 번호 : [8176]
종료 토큰 번호 : [8177]
단어 집합의 크기 : 8178


### 3. 데이터 인코딩
 - encode 메서드 이용해 문장 인코딩 진행
 - ex) 가족들이랑 어디 가지 ?  =>  [7878, 1156, 1055, 2]
 - decode 메서드 이용해 문장 디코딩 진행
 - ex) [7878, 1156, 1055, 2]  => 가족들이랑 어디 가지? 

In [101]:
# 인코딩 및 디코딩 테스트
sample = questions[32]
encoded_sample = tokenizer.encode(sample)
decoded_sample = tokenizer.decode(encoded_sample)
print('질문 샘플 :', sample)
print()
print('인코딩 과정:', sample, '->', encoded_sample)
print('디코딩 과정:', encoded_sample, '->', decoded_sample)


질문 샘플 : 가족들이랑 어디 가지 ?

인코딩 과정: 가족들이랑 어디 가지 ? -> [7878, 1156, 1055, 2]
디코딩 과정: [7878, 1156, 1055, 2] -> 가족들이랑 어디 가지 ?


In [102]:
# 인코딩 최대 길이 = 40
MAX_LENGTH = 40

# 전체 데이터를 인코딩하는 함수
def tokenize_and_filter(inputs, outputs):
    """
    토큰화 -> 인코딩 -> SOS 토큰과 EOS 토큰 추가 -> 패딩
    """
    tokenized_inputs, tokenized_outputs = [], []

    for (sentence1, sentence2) in zip(inputs, outputs):
        # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
        sentence1 = START_TKN + tokenizer.encode(sentence1) + END_TKN
        sentence2 = START_TKN + tokenizer.encode(sentence2) + END_TKN

        tokenized_inputs.append(sentence1)
        tokenized_outputs.append(sentence2)

    # 패딩
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

    return tokenized_inputs, tokenized_outputs

In [103]:
questions, answers = tokenize_and_filter(questions, answers)

In [106]:
print(type(questions))
questions.shape

<class 'numpy.ndarray'>


(11823, 40)

In [105]:
# 샘플 확인
print(questions[2])
print(answers[2])

[8176 7971 1434 4651 7952 3650   67 8177    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[8176 3398  776  131    1 8177    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


### 4. 데이터셋 생성
 - tf.data.Dataset 이용

In [107]:
batch_size = 64
BUFFER_SIZE = 20000

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1] # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
    },
    {
        'outputs': answers[:, 1:]  # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다.
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [108]:
# 임의의 샘플에 대해서 [:, :-1]과 [:, 1:]이 어떤 의미를 가지는지 테스트해본다.
print(answers[0]) # 기존 샘플
print(answers[:1][:, :-1]) # 마지막 패딩 토큰 제거하면서 길이가 39가 된다.
print(answers[:1][:, 1:]) # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다. 길이는 역시 39가 된다.

[8176 3842   74 7893    1 8177    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]
[[8176 3842   74 7893    1 8177    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
[[3842   74 7893    1 8177    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]


### 5. 모델 정의
 - 트랜스포머 모델을 불러와 사용

In [114]:
import transformer

# Hyperparameter
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1
MAX_LENGTH = 40

model = transformer.transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

In [115]:
learning_rate = transformer.CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=transformer.loss_function, metrics=[accuracy])

In [117]:
transformer.MAX_LENGTH

AttributeError: module 'transformer' has no attribute 'MAX_LENGTH'

In [116]:
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS)

Epoch 1/50


NameError: in user code:

    File "C:\Users\jaehy\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\jaehy\Desktop\github\AI\NLP\transformer.py", line 322, in loss_function  *
        y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

    NameError: name 'MAX_LENGTH' is not defined


### 6. 챗봇 구현

In [ ]:
def preprocess_sentence(sentence):
  # 단어와 구두점 사이에 공백 추가.
  # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    return sentence

In [ ]:
def evaluate(sentence):
    # 입력 문장에 대한 전처리
    sentence = preprocess_sentence(sentence)

    # 입력 문장에 시작 토큰과 종료 토큰을 추가
    sentence = tf.expand_dims(START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

    output = tf.expand_dims(START_TOKEN, 0)

    # 디코더의 예측 시작
    for i in range(MAX_LENGTH):
        predictions = model(inputs=[sentence, output], training=False)

    # 현재 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, END_TOKEN[0]):
        break

    # 현재 시점의 예측 단어를 output(출력)에 연결한다.
    # output은 for문의 다음 루프에서 디코더의 입력이 된다.
    output = tf.concat([output, predicted_id], axis=-1)

    # 단어 예측이 모두 끝났다면 output을 리턴.
    return tf.squeeze(output, axis=0)

In [ ]:
def predict(sentence):
    prediction = evaluate(sentence)

    # prediction == 디코더가 리턴한 챗봇의 대답에 해당하는 정수 시퀀스
    # tokenizer.decode()를 통해 정수 시퀀스를 문자열로 디코딩.
    predicted_sentence = tokenizer.decode([i for i in prediction if i < tokenizer.vocab_size])

    print('Input: {}'.format(sentence))
    print('Output: {}'.format(predicted_sentence))

    return predicted_sentence